<a href="https://colab.research.google.com/github/Santa7714x/Sanjay-fab7/blob/main/Classification_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import os
import shutil
import random

# Define source and destination paths
source_dir = r'D:/Y3 Internship/images/STI_LIVE01/images'
train_dir = r'D:/Y3 Internship/images/STI_LIVE01/train'
val_dir = r'D:/Y3 Internship/images/STI_LIVE01/validation'
test_dir = r'D:/Y3 Internship/images/STI_LIVE01/test'

# Create train, validation, and test directories if they don't exist
os.makedirs(train_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

# Split the images
for defect_class in os.listdir(source_dir):
    class_path = os.path.join(source_dir, defect_class)

    if os.path.isdir(class_path):
        images = os.listdir(class_path)
        random.shuffle(images)

        train_split = int(0.8 * len(images))
        val_split = int(0.9 * len(images))

        train_images = images[:train_split]
        val_images = images[train_split:val_split]
        test_images = images[val_split:]

        # Move images to train directory
        for img in train_images:
            os.makedirs(os.path.join(train_dir, defect_class), exist_ok=True)
            shutil.move(os.path.join(class_path, img), os.path.join(train_dir, defect_class, img))

        # Move images to validation directory
        for img in val_images:
            os.makedirs(os.path.join(val_dir, defect_class), exist_ok=True)
            shutil.move(os.path.join(class_path, img), os.path.join(val_dir, defect_class, img))

        # Move images to test directory
        for img in test_images:
            os.makedirs(os.path.join(test_dir, defect_class), exist_ok=True)
            shutil.move(os.path.join(class_path, img), os.path.join(test_dir, defect_class, img))

print("Data successfully split into train, validation, and test folders.")


FileNotFoundError: [Errno 2] No such file or directory: 'D:/Y3 Internship/images/STI_LIVE01/images'

In [4]:
import os
import shutil
import random

# Define source and destination paths
source_dir = r'D:/Y3 Internship/images/STI_LIVE01/images'
train_dir = r'D:/Y3 Internship/images/STI_LIVE01/train'
val_dir = r'D:/Y3 Internship/images/STI_LIVE01/validation'
test_dir = r'D:/Y3 Internship/images/STI_LIVE01/test'

# Create train, validation, and test directories if they don't exist
os.makedirs(train_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

# Split the images
for defect_class in os.listdir(source_dir):  # Iterate through defect class folders
    class_path = os.path.join(source_dir, defect_class)

    if os.path.isdir(class_path):  # Check if it's a directory (defect class)
        images = os.listdir(class_path)  # List images within the defect class folder
        random.shuffle(images)

        train_split = int(0.8 * len(images))
        val_split = int(0.9 * len(images))

        train_images = images[:train_split]
        val_images = images[train_split:val_split]
        test_images = images[val_split:]

        # Move images to train directory
        for img in train_images:
            # Create the defect class subdirectory within the train directory if it doesn't exist
            os.makedirs(os.path.join(train_dir, defect_class), exist_ok=True)
            shutil.move(os.path.join(class_path, img), os.path.join(train_dir, defect_class, img))

        # Move images to validation directory
        for img in val_images:
            # Create the defect class subdirectory within the validation directory if it doesn't exist
            os.makedirs(os.path.join(val_dir, defect_class), exist_ok=True)
            shutil.move(os.path.join(class_path, img), os.path.join(val_dir, defect_class, img))

        # Move images to test directory
        for img in test_images:
            # Create the defect class subdirectory within the test directory if it doesn't exist
            os.makedirs(os.path.join(test_dir, defect_class), exist_ok=True)
            shutil.move(os.path.join(class_path, img), os.path.join(test_dir, defect_class, img))

print("Data successfully split into train, validation, and test folders.")

FileNotFoundError: [Errno 2] No such file or directory: 'D:/Y3 Internship/images/STI_LIVE01/images'

In [ ]:
# Import necessary libraries
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
import os

# Set paths
train_dir = 'D:/Y3 Internship/images/STI_LIVE01/train'
val_dir = 'D:/Y3 Internship/images/STI_LIVE01/validation'
test_dir = 'D:/Y3 Internship/images/STI_LIVE01/test'

# Parameters
img_height, img_width = 150, 150  # Resize images for uniform input
batch_size = 32

# Automatically determine the number of classes by counting the folders
num_classes = len(next(os.walk(train_dir))[1])

# Data augmentation for training and rescaling for validation/test
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    horizontal_flip=True
)

val_test_datagen = ImageDataGenerator(rescale=1.0/255)

# Load data
train_data = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'
)

val_data = val_test_datagen.flow_from_directory(
    val_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'
)

test_data = val_test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

# Build the model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(img_height, img_width, 3)),
    MaxPooling2D(pool_size=(2, 2)),

    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),

    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')  # Dynamically set based on the number of defect classes
])

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Print model summary
model.summary()

# Train the Model
epochs = 20  # Number of epochs, adjust as needed

history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=epochs,
    steps_per_epoch=train_data.samples // batch_size,
    validation_steps=val_data.samples // batch_size
)

# Evaluate the Model on Test Data
test_loss, test_accuracy = model.evaluate(test_data)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

# Save the model
model.save('defect_classification_model.h5')
print("Model training complete and saved as 'defect_classification_model.h5'")
